In [ ]:
import os 
from langchain_google_genai import ChatGoogleGenerativeAI #using for chat model
from langchain_text_splitters import RecursiveCharacterTextSplitter #using for split text into chunks
from langchain_google_genai import GoogleGenerativeAIEmbeddings #using for embedding model
from langchain_community.vectorstores import Chroma #using for store vectore db 
from langchain_community.document_loaders import TextLoader #using for load text documents 
from langchain.chains import RetrievalQA #using for QA Chain = टीचर → उन्हीं पेजों को पढ़कर आसान भाषा में जवाब देता है 👩‍🏫📌 QA Chain कहाँ काम आता है?
#LangChain में RetrievalQA इसी स्टेप के लिए बना है। यह तीन चीज़ों को जोड़ता है: Retriever (जो text chunks निकालता है) LLM (जो answer लिखता है) Prompt Template (जो context + question को LLM के input में बदलता है)
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv               #QA Chain का इस्तेमाल यहीं होता है ✅ Retriever से आए chunks को LLM (जैसे GPT, LLaMA) में भेजा जाता है
                                            #एक Prompt Template यूज़ होता है → जिसमें context + question डाला जाता है LLM उसी context के आधार पर answer जनरेट करता है
load_dotenv() 
                                                      

True

In [2]:
#load documents 
loader=TextLoader("solar_system.txt")
docs=loader.load()

In [3]:
#split text into chunks
# 2. Split into Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20)
chunks = text_splitter.split_documents(docs)

In [4]:
# Embed and store in Chroma vector DB
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=os.environ["GOOGLE_API_KEY"])
db = Chroma.from_documents(chunks, embeddings)

In [5]:
# Initialize Gemini chat model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

In [6]:

# Build RAG chain
qa = RetrievalQA.from_chain_type(llm=llm, retriever=db.as_retriever(), chain_type="stuff")

In [7]:
# Ask a question based on the Solar System file
question = "How many planets in our solar system?"
response = llm.invoke(question)

print("Question:", question)
print("Answer:", response.content)

Question: How many planets in our solar system?
Answer: There are eight planets in our solar system.


In [ ]:
while True:
    query = input("You: ")   # 1️⃣ User se input lo
    if query.lower() in ["bye", "exit", "quit"]:   # 2️⃣ Agar exit word hai
        print("Bot: Bye 👋")                       #    Bot farewell bolega
        break                                      #    Loop tod do (program stop)
    result = qa_chain.run(query)   # 3️⃣ User query ko qa_chain ke paas bhejo
    print("Bot:", result)          # 4️⃣ Bot ka answer print karo
